# Adaptive Wellness Companion

This experience pairs conversation, tailored wellness plans, DALLE imagery, and TTS audio into a single Gradio interface.

In [3]:
# Core imports for the companion
import os
import json
import base64
from io import BytesIO
from pathlib import Path
from tempfile import NamedTemporaryFile

from openai import OpenAI
import gradio as gr
from PIL import Image
from dotenv import load_dotenv

In [4]:
# Create the OpenAI client and validate configuration
load_dotenv()
if not os.getenv("OPENAI_API_KEY"):
    raise RuntimeError("Set OPENAI_API_KEY before running the wellness companion.")

client = OpenAI()

In [5]:
# Model constants and system persona
MODEL = "gpt-4o-mini"
IMAGE_MODEL = "dall-e-3"
VOICE_MODEL = "gpt-4o-mini-tts"

system_message = (
    "You are an upbeat adaptive wellness coach. "
    "Blend evidence-backed guidance with empathy, tailor plans "
    "to the user's mood, energy, and stress, and explain reasoning concisely."
)

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_wellness_plan",
            "description": "Build a wellness micro-plan keyed to the user's current state.",
            "parameters": {
                "type": "object",
                "properties": {
                    "mood": {"type": "string", "description": "How the user currently feels."},
                    "energy": {"type": "string", "description": "Low, medium, or high energy."},
                    "stress": {"type": "string", "description": "Stress intensity words like calm or overwhelmed."},
                    "focus_goal": {"type": "string", "description": "What the user needs help focusing on right now."}
                },
                "required": ["mood", "energy", "stress", "focus_goal"]
            }
        }
    }
]

In [6]:
# Tool backends that the model can call during chat
def get_wellness_plan(mood: str, energy: str, stress: str, focus_goal: str) -> str:
    energy = energy.lower()
    stress = stress.lower()
    palette = "calming watercolor"
    movement = "gentle mobility flow"
    breathing = "box breathing (4-4-4-4)"
    journaling = "List three small wins and one supportive next step."

    if "high" in energy:
        movement = "energizing interval walk with posture resets"
        breathing = "alternate nostril breathing to balance focus"
    elif "low" in energy:
        movement = "floor-based decompression stretches"

    if "over" in stress or "anx" in stress:
        palette = "soothing pastel sanctuary"
        breathing = "4-7-8 breathing to downshift the nervous system"
    elif "calm" in stress:
        palette = "sunlit studio with optimistic accents"

    focus_goal = focus_goal.strip() or "refocus"

    plan = {
        "headline": "Adaptive wellness reset",
        "visual_theme": f"{palette} inspired by {mood}",
        "movement": movement,
        "breathing": breathing,
        "reflection": f"Prompt: {journaling}",
        "focus_affirmation": f"Affirmation: You have the capacity to handle {focus_goal} with grace."
    }
    return json.dumps(plan)

tool_registry = {"get_wellness_plan": get_wellness_plan}

In [7]:
# Multimodal helpers: text-to-speech and imagery
def talker(message: str) -> str | None:
    if not message:
        return None
    try:
        with client.audio.speech.with_streaming_response.create(
            model=VOICE_MODEL,
            voice="alloy",
            input=message
        ) as response:
            temp_file = NamedTemporaryFile(suffix=".mp3", delete=False)
            temp_path = temp_file.name
            temp_file.close()
            response.stream_to_file(temp_path)
        return temp_path
    except Exception as exc:
        print(f"[warn] audio synthesis unavailable: {exc}")
        return None

def artist(theme: str) -> Image.Image | None:
    if not theme:
        return None
    try:
        prompt = (
            f"Immersive poster celebrating a wellness ritual, {theme}, "
            "with hopeful lighting and inclusive representation."
        )
        response = client.images.generate(
            model=IMAGE_MODEL,
            prompt=prompt,
            size="1024x1024",
            response_format="b64_json"
        )
        image_base64 = response.data[0].b64_json
        image_data = base64.b64decode(image_base64)
        return Image.open(BytesIO(image_data))
    except Exception as exc:
        print(f"[warn] image generation unavailable: {exc}")
        return None

In [11]:
# Conversation orchestration with tool calls, imagery, and audio
def handle_tool_calls_and_theme(message) -> tuple[list[dict], str | None]:
    responses = []
    theme = None
    for tool_call in message.tool_calls or []:
        if tool_call.function.name not in tool_registry:
            continue
        arguments = json.loads(tool_call.function.arguments)
        result = tool_registry[tool_call.function.name](**arguments)
        responses.append(
            {"role": "tool", "tool_call_id": tool_call.id, "content": result}
        )
        payload = json.loads(result)
        theme = theme or payload.get("visual_theme")
    return responses, theme

def chat(history: list[dict]) -> tuple[list[dict], str | None, Image.Image | None]:
    conversation = [{"role": item["role"], "content": item["content"]} for item in history]
    messages = [{"role": "system", "content": system_message}] + conversation
    response = client.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    theme = None

    while response.choices[0].finish_reason == "tool_calls":
        tool_message = response.choices[0].message
        tool_responses, candidate_theme = handle_tool_calls_and_theme(tool_message)
        if candidate_theme:
            theme = candidate_theme
        messages.append(tool_message)
        messages.extend(tool_responses)
        response = client.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    reply = response.choices[0].message.content
    updated_history = history + [{"role": "assistant", "content": reply}]
    audio_path = talker(reply)
    image = artist(theme)
    print(image)
    return updated_history, audio_path, image

def put_message_in_chatbot(message: str, history: list[dict]) -> tuple[str, list[dict]]:
    return "", history + [{"role": "user", "content": message}]

In [12]:
# Assemble the Gradio Blocks UI
with gr.Blocks(title="Adaptive Wellness Companion") as wellness_ui:
    gr.Markdown("### Tell me how you are doing and I'll craft a micro-plan.")
    with gr.Row():
        chatbot = gr.Chatbot(height=420, type="messages", label="Conversation")
        image_output = gr.Image(height=420, label="Visual Inspiration")
    audio_output = gr.Audio(label="Coach Audio", autoplay=True)
    mood_input = gr.Textbox(label="Share your update", placeholder="e.g. Feeling drained after meetings")

    mood_input.submit(
        fn=put_message_in_chatbot,
        inputs=[mood_input, chatbot],
        outputs=[mood_input, chatbot]
    ).then(
        fn=chat,
        inputs=chatbot,
        outputs=[chatbot, audio_output, image_output]
    )

wellness_ui.queue()

Gradio Blocks instance: 2 backend functions
-------------------------------------------
fn_index=0
 inputs:
 |-<gradio.components.textbox.Textbox object at 0x11b3d2850>
 |-<gradio.components.chatbot.Chatbot object at 0x11b3d20d0>
 outputs:
 |-<gradio.components.textbox.Textbox object at 0x11b3d2850>
 |-<gradio.components.chatbot.Chatbot object at 0x11b3d20d0>
fn_index=1
 inputs:
 |-<gradio.components.chatbot.Chatbot object at 0x11b3d20d0>
 outputs:
 |-<gradio.components.chatbot.Chatbot object at 0x11b3d20d0>
 |-<gradio.components.audio.Audio object at 0x11b3d25d0>
 |-<gradio.components.image.Image object at 0x11b3d2210>

In [10]:
# Launch the interface inline when running in a notebook
wellness_ui.launch(inline=True, share=False, prevent_thread_lock=True)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
